In [ ]:
import pandas as pd,torch,numpy as np,math

num_question=20
sample_per_question=5
!pip install datasets
from datasets import load_dataset


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00


In [ ]:
def check_entailment(question,premise,llm_res):
  from transformers import AutoModelForSequenceClassification,AutoTokenizer
  #checkpoint='microsoft/deberta-v2-xlarge-mnli'
  checkpoint='microsoft/deberta-base-mnli'
  tokenizer=AutoTokenizer.from_pretrained(checkpoint)
  model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
  ##but what is in the question var
  formatted_premise=f'Here are the question and premise--Question:{question}'+f' {premise}'
  formatted_response=f'This is the response: {llm_res}'
  input=tokenizer(formatted_premise,formatted_response,padding=True,return_tensors='pt')
  with torch.no_grad():
         output=model(**input)
       #print(f"Here come the logits for Question {i} using response{j}: \n\n")
  logits=output.logits
  proba=torch.nn.functional.softmax(output.logits)
  return model,proba

In [ ]:
def Do_NLI(dataset, samples):
    # Assuming cluster_df is already defined as a DataFrame somewhere in your code
    cluster_df = pd.DataFrame(columns=['Premise', 'Hypothesis', 'Entailment'])
    premises=[]
    prediction=[]
    step=int(len(dataset)/num_question)
    '''Entailing each response to a question to each other'''
    for i in range(0,step):
      for j in range(0,sample_per_question):
        #premise=samples['LLM Response'][5*i + j]
        if num_question*i + j==100:
          print(num_question)
          print(i)
          print(j)
        premise=samples['LLM Response'][num_question*i + j]
        premises.append(premise)
        for z in range(0,sample_per_question):
          #try:
          #llm_res=samples['LLM Response'][5*i +(z)]
          llm_res=samples['LLM Response'][num_question*i +(z)]
          '''except Exception as e:
          break
          '''
          model,proba=check_entailment(samples['Question'][i],premise,llm_res )
          label=model.config.id2label[int(torch.argmax(proba))]
          prediction.append(label)
          # Construct new row as DataFrame
          new_row = pd.DataFrame({'Premise': [premise], 'Hypothesis': [llm_res], 'Entailment': [label]})
          cluster_df = pd.concat([cluster_df, new_row], ignore_index=True)

    return cluster_df


In [ ]:
def cluster_data(NLI_data):
      #print(num_question)
      data=NLI_data
      print(len(data['Entailment']))
      clusters=dict()
      step=int(len(NLI_data)/num_question)
      id=num_question * sample_per_question
      for i in range(0,num_question):
          start=sample_per_question*i
          end=((num_question * sample_per_question)*i)+num_question * sample_per_question
          print(f"The start is {start} and end is {end} ")
          data=data[start:end]
          #data=data[sample_per_question*i:((num_question * sample_per_question)*i)+num_question * sample_per_question]
          #print(data.head())
          if f'que{i}' not in clusters:
                  clusters[f'que{i}']={}
                  #clusters.update({f'clusters{i}':[]})
      print(data['Entailment'])
      for j in range(0,step):
         for k in range(0,step):
              if f'sample{j}' not in clusters[f'que{i}'].keys():
                    clusters[f'que{i}'][f'sample{j}']=[]
              print(f'Current index is {(num_question*j)+k} ')
              try:
                if data['Entailment'].iloc[(num_question*j)+k]=='ENTAILMENT':
                    clusters[f'que{i}'][f'sample{j}'].append(k+1)
              except Exception as e:
                print((num_question*j)+k)
                print(f"j is {j}")
                print(f"k is {k}")

      return clusters

In [ ]:
#group_clusters(clusters)
def group_clusters(clusters):
    all_clust=np.arange(1,sample_per_question+1)
    for key in clusters.keys():
        val = []
        # Append all sample lists to val for comparison
        for sample_key in clusters[key].keys():
            val.append(clusters[key][sample_key])

        # Compare all combinations of the samples
        for i in range(len(val)):
            for j in range(i + 1, len(val)):
                # Compare two different samples
                if set(val[i]) == set(val[j]):
                    # Remove sample if it's identical to another
                    try:
                        del clusters[key][f'sample{j}']
                    except Exception as e:
                        continue
    '''if certain numbers are not in a clusters make them them into individual clusters'''
    #ref=list(clusters[question].values())
    for valu in all_clust:
        for question in clusters.keys():
          print(question)
          ref = [item for sublist in clusters[question].values() for item in sublist]
          if valu in ref:
              continue
          else:
              #print(valu)
              #print(values)
              clusters[question].update({f'n_sample{valu}':[valu]})


    return clusters

In [ ]:
def semantic_entropy(clusters):
  SE=[]
  for question in clusters.keys():
    #print(question)
    sb=[]
    for response in clusters[question].keys():
      #print(clusters[question][response])
      if len(clusters[question][response])==0:
        pc=1/sample_per_question
      else:
        pc=len(clusters[question][response])/sample_per_question
      sb.append(-pc*math.log2(pc))
    SE.append(sum(sb))
  return SE

In [ ]:
#start with western questions English
triviaQA=load_dataset('mandarjoshi/trivia_qa','rc')
western_questions=triviaQA['train'].select(range(20))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

train-00000-of-00026.parquet:   0%|          | 0.00/308M [00:00<?, ?B/s]

...

test-00003-of-00004.parquet:   0%|          | 0.00/128M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
#question=pd.read_xlsx(data)
data_responses=pd.read_excel('/content/Hal_Varying_Temperature_NEW.xlsx')
data_responses=data_responses.iloc[0:20].values.tolist()
data_responses=pd.DataFrame(data_responses,columns=['Question','LLM Response','Temperature','Seed'])

In [ ]:
NLI_data=Do_NLI(western_questions,data_responses)
NLI_data

In [ ]:
clusters=cluster_data(NLI_data)
clusters

In [ ]:
grp_clusters=group_clusters(clusters)
grp_clusters

In [ ]:
hallucination=semantic_entropy(grp_clusters)
hallucination

In [ ]:
results=pd.DataFrame(columns=['Question','Semantic Entropy'])
for i in range(0,len(hallucination)):
  resul=pd.DataFrame({'Question':[disp['Question'][sample_per_question * i]], 'Semantic Entropy':[hallucination[i]]})
  results=pd.concat([results,resul],ignore_index=True)
results